<a href="https://colab.research.google.com/github/dale108/AD340/blob/master/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import os
import zipfile
import tensorflow as tf
import numpy as np
from google.colab import files
from keras.preprocessing import image
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from google.colab import files
import glob

In [73]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O cats_and_dogs_filtered.zip

--2022-04-08 05:27:40--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 142.251.18.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   154MB/s    in 0.4s    

2022-04-08 05:27:41 (154 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [74]:
dogs_zip = '/content/cats_and_dogs_filtered.zip'
new_dogs = zipfile.ZipFile(dogs_zip, 'r')
new_dogs.extractall('/content')
new_dogs.close()

coyotes_zip = '/content/coyote_images.zip'
new_coyotes = zipfile.ZipFile(coyotes_zip, 'r')
new_coyotes.extractall('/content')
new_coyotes.close()

In [75]:
base_directory = '/content/'
train_directory = os.path.join(base_directory, 'train')
validation_directory = os.path.join(base_directory, 'test')
print(validation_directory)

/content/test


In [76]:
train_coyotes_directory = os.path.join(train_directory, 'coyotes')
train_dogs_directory = os.path.join('/content/cats_and_dogs_filtered/train', 'dogs')
validation_coyotes_directory = os.path.join(validation_directory, 'coyotes')
validation_dogs_directory = os.path.join('/content/cats_and_dogs_filtered/validation', 'dogs')

In [77]:
n_rows = 4
n_cols = 4
index = 0

In [78]:
train_coyote_names = os.listdir(train_coyotes_directory)
train_dog_names = os.listdir(train_dogs_directory)
print(train_coyote_names[:3])
print(train_dog_names[:3])

['image790.png', 'image365.png', 'image289.png']
['dog.56.jpg', 'dog.675.jpg', 'dog.465.jpg']


In [79]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

In [80]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 64)      

In [81]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [82]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_directory,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))

validation_generator =  test_datagen.flow_from_directory( validation_directory,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 469 images belonging to 2 classes.
Found 264 images belonging to 2 classes.


In [83]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=100,
    epochs=50,
    validation_steps=50)

UnidentifiedImageError: ignored